In [31]:
# General libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# For preprocessing
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from skimage.transform import resize
from skimage.color import rgb2lab, lab2rgb, rgb2gray
# For NN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, UpSampling2D
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
# For output image
from skimage.io import imsave

%matplotlib inline

In [16]:
# Get training images
train_input_files = []
train_input_dir = "Images/Train/"

for (dirpath, dirnames, filenames) in os.walk(train_input_dir):
    train_input_files.extend(filenames)
    break

X = []

for filename in train_input_files:
    X.append(img_to_array(load_img(train_input_dir + filename)))

X = np.array(object = X, dtype = float)

print("X shape:", X.shape, type(X))


X shape: (9294, 256, 256, 3) <class 'numpy.ndarray'>


In [77]:
# Get test images
colour_me_files = []
colour_me_dir = "Images/Test/"

for (dirpath, dirnames, filenames) in os.walk(colour_me_dir):
    colour_me_files.extend(filenames)
    break
    
colour_me = []

for filename in colour_me_files:
    colour_me.append(img_to_array(load_img(colour_me_dir + filename)))

colour_me = np.array(object = colour_me, dtype = float)
colour_me = rgb2lab(1.0/255 * colour_me)[:, :, :, 0]
colour_me = colour_me.reshape(colour_me.shape + (1, ))

print("colour_me shape:", colour_me.shape, type(colour_me))

colour_me shape: (494, 256, 256, 1) <class 'numpy.ndarray'>


In [20]:
# Set up training data
split = int(0.95 * len(X))
X_train = X[:split]
X_train = (1.0 / 255) * X_train

print("X_train shape:", X_train.shape, type(X_train))

X_train shape: (8829, 256, 256, 3) <class 'numpy.ndarray'>


In [68]:
# Set up test data
X_test = X[split:]
X_test = rgb2lab((1.0 / 255) * X_test)
X_test = X_test[:, :, :, 0]
X_test = X_test.reshape(X_test.shape + (1, ))
print("X_test shape:", X_test.shape, type(X_test))

Y_test = rgb2lab(1.0/255 * X[split:])
Y_test = Y_test[:, :, :, 1:]
Y_test /= 128
print("Y_test shape:", Y_test.shape, type(Y_test))

X_test shape: (465, 256, 256, 1) <class 'numpy.ndarray'>
Y_test shape: (465, 256, 256, 2) <class 'numpy.ndarray'>


In [27]:
# Set up training sample data of 100 images
X_train_sample = X_train[np.random.choice(X_train.shape[0], size = 100, replace = False)]

print("X_train_sample shape:", X_train_sample.shape, type(X_train_sample))

X_train_sample shape: (100, 256, 256, 3) <class 'numpy.ndarray'>


In [61]:
# Building the neural network

model = Sequential(name = "Colouring-FashionMNIST")
model.add(InputLayer(input_shape = (256, 256, 1)))
model.add(Conv2D(name = "Conv2D_8a_3x3", filters = 8, kernel_size = (3, 3), activation = 'relu', padding = 'same', strides = 2))
model.add(Conv2D(name = "Conv2D_8b_3x3", filters = 8, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(name = "Conv2D_16a_3x3", filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(name = "Conv2D_16b_3x3", filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same', strides = 2))
model.add(Conv2D(name = "Conv2D_32a_3x3", filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(name = "Conv2D_32b_3x3", filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same', strides = 2))
model.add(UpSampling2D(name = "UpSamp-a_2x2", size = (2, 2)))
model.add(Conv2D(name = "Conv2D_32c_3x3", filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(UpSampling2D(name = "UpSamp-b_2x2", size = (2, 2)))
model.add(Conv2D(name = "Conv2D_16c_3x3", filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(UpSampling2D(name = "UpSamp-c_2x2", size = (2, 2)))
model.add(Conv2D(name = "Conv2D_2_3x3", filters = 2, kernel_size = (3, 3), activation = 'tanh', padding = 'same'))


In [62]:
model.compile(optimizer = 'rmsprop', loss = 'mse')

print(model.summary())

Model: "Colouring-FashionMNIST"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_8a_3x3 (Conv2D)       (None, 128, 128, 8)       80        
_________________________________________________________________
Conv2D_8b_3x3 (Conv2D)       (None, 128, 128, 8)       584       
_________________________________________________________________
Conv2D_16a_3x3 (Conv2D)      (None, 128, 128, 16)      1168      
_________________________________________________________________
Conv2D_16b_3x3 (Conv2D)      (None, 64, 64, 16)        2320      
_________________________________________________________________
Conv2D_32a_3x3 (Conv2D)      (None, 64, 64, 32)        4640      
_________________________________________________________________
Conv2D_32b_3x3 (Conv2D)      (None, 32, 32, 32)        9248      
_________________________________________________________________
UpSamp-a_2x2 (UpSampling2D)  (None, 64, 64, 

In [63]:
# Image transformer

datagen = ImageDataGenerator(shear_range = 0.2,
                             zoom_range = 0.2,
                             rotation_range = 20,
                             horizontal_flip = True)

# ImageDataGenerator returns a transformed version of the input (same shape)

In [64]:
# Generate training data
batch_size = 50

def image_a_b_gen(batch_size):
    
    '''Note that this returns a tuple of (inputs, targets),
    where inputs is a np.array with shape (batch_size, h, w, 1)
    and targets is a np.array with shape (batch_sze, h, w, 2).'''
    
    for batch in datagen.flow(X_train, batch_size = batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:, :, :, 0]
        Y_batch = lab_batch[:, :, :, 1:]
        Y_batch /= 128
        yield (X_batch.reshape(X_batch.shape + (1, )), Y_batch)


In [65]:
# Train the model
# TensorBoard(log_dir = "/Beta_Outputs")

model.fit(image_a_b_gen(batch_size),
          steps_per_epoch = 100, # Setting steps_per_epoch < len(trainging_data) // batch_size ensures that we don't run through the entire training data
          epochs = 1,
          verbose = 1)


100/100 [==============================] - 237s 2s/step - loss: 0.0108


In [69]:
# Test the model
print(model.evaluate(X_test, Y_test, batch_size = batch_size))

10/10 [==============================] - 3s 292ms/step - loss: 0.0109
0.010871941223740578


In [78]:
# Test the model on colour_me
output = model.predict(colour_me)
output *= 128

In [79]:
# Output colourisations of colour_me
for i in range(len(output)):
    output_array = np.zeros((256, 256, 3))
    output_array[:, :, 0] = colour_me[i][:, :, 0]
    output_array[:, :, 1:] = output[i]
    imsave(f"Beta_Outputs/img_{i}.png", lab2rgb(output_array))

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0